# Restack 20km AK WRF data interactively

This notebook is for ad-hoc execution of any portion of the pipeline in the [restack_20km](./restack_20km.ipynb) notebook. Use it to restack arbitrary chunks of the WRF data, down to a single variable of a single year for a particular WRF group.

This notebook assumes that:
* all potential target files have been copied to the scratch directory
* the forecast times table has been created per step 1 in the [restack_20km](./restack_20km.ipynb) notebook

## Setup

Execute this cell to set up the environment:

In [8]:
import os
import tqdm
# codebase
from config import *
import luts
import slurm
import restack_20km as main

## Restack data


### Processing parameters

The following arguments are required for a single job of restacking data for a particular variable (or variables), model, scenario, and year (or years):

* **WRF Group**: Encoded value specifying the WRF group being worked on, which should already be specified in the `WRF_GROUP` environment variable prior to starting this notebook.
* **Year(s)**: a list of years to work on specified as integers, such as `[1979, 1980]`, or omit to work on all years available for a given WRF group.
* **Variable name(s)**: Name(s) of the variable(s). This is the lower case version of the variable name in the WRF outputs.
* **Number of cores**: This is the number of cores to use for parallel tasks.
* **Chinook partition**: the desired partition on the Chinook cluster.

In [17]:
years = [1738]
valid_years = f"{luts.groups[group]['years'][0]}-{luts.groups[group]['years'][-1]}"
while not all([int(year) in luts.groups[group]["years"] for year in years]):
    years = input(f"Years, a ' '- separated list of years, in {valid_years}. Leave blank for all years:") or luts.groups[group]['years']
    if (type(years) == str) and (len(years) > 0):
        years = years.split(" ")
print(f"Years selected: {years}")

varnames = [""]
while not all ([varname.upper() in luts.varnames for varname in varnames]):
    varnames = input("Enter name(s) of WRF variable(s) to restack (leave blank for all):") or luts.varnames
    if (type(varnames) == str) and (len(varnames) > 0):
        varnames = [varname.upper() for varname in varnames.split(" ")]

ncpus = 0
while (ncpus < 2) | (ncpus > 24):
    ncpus = input("Enter number of CPUs to use (valid range: 2-24; leave blank for 24 cores):") or 24
    ncpus = int(ncpus)

partition = input("Enter name of compute partition to use (leave blank for 't1small'):") or "t1small"

Years, a ' '- separated list of years, in 1970-2005. Leave blank for all years: 1972


Years selected: ['1972']


Enter name(s) of WRF variable(s) to restack (leave blank for all): qbot
Enter number of CPUs to use (valid range: 2-24; leave blank for 24 cores): 
Enter name of compute partition to use (leave blank for 't1small'): 


Create slurm scripts:

In [18]:
sbatch_fps = []
year_str = main.get_year_fn_str(years)
for varname in varnames:
    # write to .slurm script
    sbatch_fp = slurm_dir.joinpath(f"restack_{group}_{year_str}_{varname}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = slurm_dir.joinpath(f"restack_{group}_{year_str}_{varname}_%j.out")
    sbatch_head = slurm.make_sbatch_head(
        slurm_email, partition, conda_init_script
    )
    
    args = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "restack_script": restack_script,
        "luts_fp": luts_fp,
        "geogrid_fp": geogrid_fp,
        "anc_dir": anc_dir,
        "restack_dir": hourly_dir,
        "group": group,
        "fn_str": luts.groups[group]["fn_str"],
        "years": years,
        "varname": varname,
        "ncpus": ncpus,
        "sbatch_head": sbatch_head,
    }

    slurm.write_sbatch_restack(**args)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output scripts if you fancy it:

In [23]:
for varname in varnames:
    _ = [fp.unlink() for fp in list(slurm_dir.glob(f"restack_{group}_{year_str}_{varname}_*.out"))]

Submit the `.slurm` scripts with `sbatch`:

In [25]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]